In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

tf.__version__

'1.6.0'

## Load data

In [2]:
bmi_df = pd.read_csv('data/bmi.csv')
bmi_df.head()

,height,weight,label
0,125,50,fat
1,189,60,thin
2,181,53,thin
3,138,36,normal
4,149,68,fat


In [3]:
bmi_df.describe()

,height,weight
count,20000.000000,20000.000000
mean,160.060800,57.613250
std,23.259842,13.220337
min,120.000000,35.000000
25%,140.000000,46.000000
50%,160.000000,58.000000
75%,180.000000,69.000000
max,200.000000,80.000000


## Preprocess data

In [4]:
print(bmi_df.label.value_counts().index)  # ['fat', 'thin', 'normal']

bmi_df.height = bmi_df.height / 200
bmi_df.weight = bmi_df.weight / 100
bclass = {'thin':[1, 0, 0], 'normal':[0, 1, 0], 'fat':[0, 0, 1]}
bmi_df['label_pat'] = bmi_df.label.apply(lambda x : np.array(bclass[x]))

bmi_df.head()

Index(['fat', 'thin', 'normal'], dtype='object')


,height,weight,label,label_pat
0,0.625,0.50,fat,"[0, 0, 1]"
1,0.945,0.60,thin,"[1, 0, 0]"
2,0.905,0.53,thin,"[1, 0, 0]"
3,0.690,0.36,normal,"[0, 1, 0]"
4,0.745,0.68,fat,"[0, 0, 1]"


In [5]:
bmi_df.describe()

,height,weight
count,20000.000000,20000.000000
mean,0.800304,0.576132
std,0.116299,0.132203
min,0.600000,0.350000
25%,0.700000,0.460000
50%,0.800000,0.580000
75%,0.900000,0.690000
max,1.000000,0.800000


In [6]:
bmi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   height     20000 non-null  float64
 1   weight     20000 non-null  float64
 2   label      20000 non-null  object 
 3   label_pat  20000 non-null  object 
dtypes: float64(2), object(2)
memory usage: 625.1+ KB


## Keep test data

In [7]:
test_csv = bmi_df[15000:20000]
test_pat = test_csv[['height', 'weight']]
test_ans = list(test_csv['label_pat'])

## Generate a graph (model)

In [8]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 2])
y = tf.placeholder(dtype=tf.float32, shape=[None, 3])

W = tf.Variable(tf.zeros([2, 3]))
b = tf.Variable(tf.zeros([3]))

h = tf.nn.softmax( tf.matmul(x, W) + b )

cross_entropy = -tf.reduce_sum( y * tf.log(h) )
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cross_entropy)

predict = tf.equal( tf.argmax(h, 1), tf.argmax(y, 1) )
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

## Train the model

In [9]:
sess = tf.Session()
sess.run( tf.global_variables_initializer() )

for step in range(5000):
    idx = (step * 100) % 14000
    
    train_csv = bmi_df[ idx : idx+100 ]  # batch_size = 100
    train_pat = train_csv[['height', 'weight']]
    train_ans = list(train_csv['label_pat'])
    sess.run([train], feed_dict={x:train_pat, y:train_ans})
    
    if step % 1000 == 0 or step > 4997:
        tmp_cost = sess.run( cross_entropy, feed_dict={x:train_pat, y:train_ans} )
        tmp_acc = sess.run( accuracy, feed_dict={x:test_pat, y:test_ans} )
        print(f'step : {step:5}  |  cost : {tmp_cost:10.5f}  |  acc : {tmp_acc:6.4f}')

step :     0  |  cost :  106.39891  |  acc : 0.3142
step :  1000  |  cost :   42.46990  |  acc : 0.8668
step :  2000  |  cost :   34.78955  |  acc : 0.9376
step :  3000  |  cost :   29.33917  |  acc : 0.9676
step :  4000  |  cost :   28.51589  |  acc : 0.9590
step :  4998  |  cost :   26.69110  |  acc : 0.9696
step :  4999  |  cost :   22.63150  |  acc : 0.9734


## Evaluate the model

In [10]:
W_final, b_final, acc = sess.run( [W, b, accuracy], feed_dict={x:test_pat, y:test_ans} )
print( f'\nfinal Weight (local optima) : \n', W_final )
print( f'\nfinal bias (local optima)   : \n', b_final )
print('\nfinal test accuracy =', acc)


final Weight (local optima) : 
 [[ 25.633936    1.3487692 -26.982683 ]
 [-20.20653     0.8207336  19.385841 ]]

final bias (local optima)   : 
 [-9.87117    -0.38069925 10.251905  ]

final test accuracy = 0.9734
